# Exercise Sheet 7

Willy Oesterheld (508791)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd


In [ ]:
# pdf
def osc_prop(energy, cos_theta, sin2theta=0.9, dm2=2.4e-3):
    l = 12e3 * cos_theta # base line - max corresponds to diameter of the Earth 12e3 km
    return 1-sin2theta * np.sin(1.27*dm2*l/energy)

# code to generate sample data taken from example notebook

def atmo_event_rate(energy, cos_theta, sin2theta=0.9, dm2=2.4e-3):
    flux = 1e5*energy**-1.7 # the true neutrino flux is steeper, this takes into account 
                            # neutrino cross-section, which is proportional to E
                            # as well as that we are working with logarithmic energy bins    
    # two flavor oscillation propability for muon disappearance - see wikipedia 
    return flux*osc_prop(energy, cos_theta, sin2theta, dm2)

def run_experiment(energy, cosdec, sin2theta=0.9, dm2=2.4e-3):
    x = np.array(energy.size)
    events=np.zeros(energy.size*cosdec.size)
    i = 0
    for e in energy:
        for c in cosdec:
            mu = atmo_event_rate(e,c,sin2theta,dm2)
            sig = np.sqrt(mu)
            event_random = np.random.normal(mu,sig)
            events[i] = int(event_random)
            i=i+1             
    return events

In [ ]:
N = 10

# lets generate one experiment, with 10 energy bins and 10 cos(declination) bins
cosdec =  np.linspace(0, 1, N) # 10 steps in cos(declination) from 0 to 1 
log10_energy = np.linspace(1, 2, N) # 10 steps in log10(energy) from 10 to 100 GeV
energy=10**log10_energy

# run the experiment 
events=run_experiment(energy,cosdec)

# plot events over angle for each energy
i = 0
fig, ax = plt.subplots(nrows=N, ncols=1, figsize=(5,N*8/10), sharex=True)
fig.subplots_adjust(hspace=0)
for e in energy:
    x = cosdec
    y = events[(N*i):(N*(i+1))]
    ax[i].plot(x, y, label=f"{e}")
    i += 1

# plot events over energy for each angle
i = 0
fig, ax = plt.subplots(nrows=N, ncols=1, figsize=(5,N*8/10), sharex=True)
fig.subplots_adjust(hspace=0)
for e in energy:
    x = energy
    y = events[i::N]
    ax[i].plot(x, y, label=f"{e}")
    ax[i].semilogx(base=10)
    i += 1


In [ ]:
X = np.zeros((N*N,2))
for i in range(0,N):
    for j in range(0,N):
        X[i*N+j,0] = energy[i]
        X[i*N+j,1] = cosdec[j]

def neg_log_likelyhood(params, X):
    sin2theta, dm2 = params
    y = atmo_event_rate(energy=X[:,0], cos_theta=X[:,1], sin2theta=sin2theta, dm2=dm2)
    return -np.sum(np.log(y))

fit = sp.optimize.minimize(neg_log_likelyhood, x0=(0.5, 2e-3), args=X, method='Nelder-Mead')
print(fit)
